# Herraminetas de NCBI

The National Center for Biotechnology Information (NCBI) es una de las colecciones de bases de datos mas importantes. Contiene secuencias de DNA, proteínas, datos taxonómicos, publicaciones etc. 

## Búsqueda de secuencais

### Por explorador
Primero vamos a explorar la base de datos de NCBI con "clicks" y después los haremos desde un script de python. Entre a https://www.ncbi.nlm.nih.gov/ y despliegue la pestaña "All Databases" para ver las opciones.

Escoja la base de datos "Protein" y escriba en la barra "Cytochrome B Homo sapiens neanderthalensis". Escoja la proteína de referencia (RefSeq Proteins) y copie el ID de esta proteína


### Desde script de python

Ahora vamos a realizar la búsqueda por ID desde python

In [ ]:
#!/usr/local/bin/

# Script por Laura Salazar Jaramillo
# Busca la secuencia de proteinas del citocromo B de neanderthales


from Bio import Entrez
from Bio import SeqIO

Entrez.email = "lsalazar@eafit.edu.co"

with Entrez.efetch(
    db="protein", rettype="fasta", retmode="text", id="YP_002124314.2") as handle:
    seq_record = SeqIO.read(handle, "fasta")

print(seq_record)
SeqIO.write(seq_record, "/home/lsalazarj/bicomp2024/ncbi_blast/hsneanderthal_cytb_prot.fa", "fasta")

## Basic Local Alignment Search Tool (BLAST)

La búsqueda de similitudes es un primer paso para encontrar  información útil sobre funcionalidad y/o ancestría. 

A la hora de buscar similitud en bases de datos extensas, los algoritmos de programación dinámica (como Needleman-Wunsch y Smith-Waterman) son ineficientes. Las herramientas mas eficientes son heurísticas: "atajos" algoritmicos que no producen una solición óptima global, pero que son mucho más rápidos y eficientes, como BLAST (Altschul et al. 1997; Altschul et al. 1990).

Por ejemplo, vamos a realizar la búsqueda de las secuencias mas similares de todas las disponibles en la base de datos usando BLAST.Estos algoritmos buscan alinear fragmentos cortos de una secuencia problema (*query*) a secuencias en una base de datos. El alineamiento inicial tiene un score T, y de allí se extiende a pequeños alineamientos vecinos. El alineamiento continúa mientras el T aumente. 

### Repaso de conceptos clave de similitud

- **Secuencias homólogas**: similitud por ancestría común
- **Alineamiento** global vs local: ordenamiento de secuencias según similitud. **Global**: utiliza toda la secuencia. **Local**: utiliza los fragmentos más similares
- **Secuencias conservadas**: aquellas que comparten nucleótidos o aminoacidos en la misma posición (opuesto en divergentes)
- **Score**: parámetro que indica la calidad del alineamiento
- **Dendograma** (Cladograma): hipótesis sobre relación evolutiva de secuencias

### E-value (Expectation-values)

Cuando realizamos una búsqueda por similitud, es importante identificar si la secuencia target es significativamente similar a la query. Para esto se utiliza un valor de significancia que se define por el score del alineamiento (S), cuando éste 
 es mayor que uno esperado al azar. Un hit siempre va a tener algún tipo de similitud, y la probabilidad de encontrar un score alto aumenta con una secuencia query mas larga (m) y/o una base de datos mas grande (n). Para cuantificar la significancia de un hit, calculamos el E-value (E), que representa el número de hits esperados con un score mínimo de *S*, usando una secuencia query de longitud *m*, y una búsqueda aleatoria en una base de datos n: 

$$
 E = K * m *n * e^{lamda S}
$$

## Tipos de BLAST

### BLAST Por explorador
Para una guía compelta sobre BLAST, consulte el [manual](https://www.ncbi.nlm.nih.gov/books/NBK1734/). 
Vamos a entrar a https://blast.ncbi.nlm.nih.gov/Blast.cgi. Hay varios tipos de programas BLAST, pero en términos generales, para bases de datos de proteínas se utiliza blastp y blastx, mientras que para nucleótidos se utiliza blastn.

Vamos a buscar las proteínas homólogas al citocromo B que descargamos. 

1. Entre a https://blast.ncbi.nlm.nih.gov/Blast.cgi y escoja la opción "Protein Blast" (porque nuestra secuencia es una proteína)
2. En la casilla en blanco pegue el fragmento de la secuencia descargada del Citocromo B de *H. sapiens neanderthalensis*
3. Deje los parámetros por defecto y de click en "BLAST".
4. El resultado tabular que aparece lista los mejores hits (de toda la base de datos, si no hubo filtros)
5. En la pestaña "Graphic Summary" verá una representación gráfica del alineamiento, donde el color rojo  muestra los *matches*
6. En la pestaña "Alignment" encontrará el alineamiento de la secuencia *query* (la que nosotros escogimos) con los mejores hits y la información del alineamiento

### BLAST remoto desde la línea de comando

Ahora vamos a realizar la búsqueda desde Apolo por medio de la línea de comando. Entre a Apolo y cargue el módulo de blast:

In [ ]:
module load blast/2.13.0_gcc-11.2.0

El comando para correr blast es (recuerde que debe estar dentro de un script slurm)

In [ ]:
blastp -query homosapiens_cytb.fa -remote -db nr -out cytb_neander_nr.txt -evalue 1e-20 -max_target_seqs 10

donde: 
- blastp: tipo de blast (proteínas en este caso)
- query: la secuencia problema
- remote: remota (de la web)
- nr: no-redundante (las secuencias idénticas están presentes usa sola vez)
- out: nombre del archivo de output
- evalue: el máximo evalue 
- max_target_seqs: un máximo de secuencias para el putput

### BLAST local

Con frecuencia estamos interesados en comparar nuestra(s) secuencia(s) problema(s) contra una base de datos específica y mas pequeña. NCBI tiene una gran colección de bases de datos descargables, las cuales se pueden descargar desde ftp://ftp.ncbi.nlm.nih.gov/blast/db/. 
Por ejemplo, nosotros podemos comparar la secuencia de citocromo B de neanderthalensis contra la base de datos de secuencias de vertebrados que habíamos trabajado antes. Para eso vamos a copiar el archivo con dichas secuencias al directorio donde estamos trabajando (o le damos la ruta absoluta en el script). Para generar la base de datos, utilizamos el siguiente comando (recuerde que debe estar dentro de un script slurm)

In [ ]:
makeblastdb -in CytBProt.txt -title cytb_vert -dbtype prot -out cytb -parse_seqids

- in:  archivo de input utilizado para generar la base de datos
- título: le da un título a la base de datos
- dbtype: si es proteína (prot) o nucleótido (nucl)
- out: Nombre de la base de datos
- parse_seqids: retiene los nombre completos de las secuencias

Observe el directorio donde se corrió el comando. Si fue exitoso, se crearon varios archivos con prefijo "cytb". Estos archivos son binarios (no son archivos de texto) y por lo tanto no se pueden inspeccionar. Ahora vamos a utilizar esta base de datos para alinear nuestra secuencia query:

In [ ]:
blastp -query hsneanderthal_cytb_prot.fa -db cytb -out cytb_blastp_1e-30_table.txt -evalue 1e-30 -outfmt 6

aquí la opción "-outfmt 6" indica el formato de output que queremos. Consulte la tabulación de este formato:
https://www.metagenomics.wiki/tools/blast/blastn-output-format-6 y revise el output que obtuvo del blast. Cual es el procentaje de similitud de esta secuencia con humanos? Si no es 100%, cómo averiguaría donde están las diferencias? 

<div class="alert alert-block alert-info">
<b>Ejercicio</b> 
Supongamos que debemos diseñar unos primers para identificar H. neanderthalensis. Como somos humanos y nuestras secuencias son altamente similares, estos primers deben tener suficiente especificidad para discriminar entre las dos especies. Cómo identificaría una posición idóndea para diseñar los primers? Cómo verificaría que estos primers que diseño sí amplifiquen *H. neanderthalensis* (que tengan sensibilidad) pero no amplifiquen humans (especificidad). 
Puede diseñar los primers manualmente o ayudarse con la herramienta primerBlast: https://www.ncbi.nlm.nih.gov/tools/primer-blast/
</div>